In [ ]:
!pip install -q datasets transformers accelerate

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
d1_url = "https://www.kaggle.com/competitions/induction-task/data"
od.download(d1_url)

In [ ]:
!pip install evaluate

In [ ]:
from evaluate import load

metric = load("accuracy")


In [ ]:
dataset_dir_1 = '/content/induction-task/Data/Train'

In [ ]:
from datasets import load_dataset
ds = load_dataset("imagefolder", data_dir=dataset_dir_1)
ds = ds['train']

In [ ]:
ds

In [ ]:
data = ds.train_test_split(test_size=0.15)
data

In [ ]:
labels = data['train'].features['label']
labels

In [ ]:
labels = data["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [ ]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-large-patch16-224'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

In [ ]:
print(feature_extractor.size)

In [ ]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
train_transforms = Compose(
        [
            RandomResizedCrop((feature_extractor.size['height'], feature_extractor.size['width'])),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize((feature_extractor.size['height'], feature_extractor.size['width'])),
            CenterCrop((feature_extractor.size['height'], feature_extractor.size['width'])),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [ ]:
train_ds = data['train']
val_ds = data['test']

In [ ]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [ ]:
from transformers import ViTForImageClassification
model_name_or_path = 'google/vit-large-patch16-224'
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    'vikas117/finetuned-ai-real-cifake',
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=True,
  report_to='tensorboard',
  load_best_model_at_end=True,
  hub_strategy="end"
)

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [ ]:
train_results = trainer.train()

trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

In [ ]:
import opendatasets as od
d1_url = "https://www.kaggle.com/competitions/induction-task-2025"
od.download(d1_url)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from PIL import Image
from transformers import AutoModelForImageClassification, AutoFeatureExtractor

repo_name = "vikas117/finetuned-ai-real-beit"

feature_extractor = AutoFeatureExtractor.from_pretrained(repo_name)
model = AutoModelForImageClassification.from_pretrained(repo_name)

In [ ]:
from torchvision import transforms
from PIL import Image
import os
import matplotlib.pyplot as plt
import torch

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
])

test_folder = "/content/induction-task-2025/Test_Images"
image_files = [os.path.join(test_folder, f) for f in os.listdir(test_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]

def load_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    return image


In [ ]:
def predict(image):
    with torch.no_grad():
        outputs = model(image)
        _, predicted_class = torch.max(outputs.logits, 1)
        return predicted_class.item()

for image_path in image_files:
    image = load_image(image_path)
    predicted_class = predict(image)

    img = Image.open(image_path)
    plt.imshow(img)
    plt.title(f"Predicted Class: {predicted_class}")
    plt.axis("off")
    plt.show()


In [ ]:
import pandas as pd
data = []

for image_file in image_files:

    image_path = os.path.join(test_folder, image_file)

    image = Image.open(image_path)
    image = image.convert("RGB")

    inputs = feature_extractor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        _, predicted_class = torch.max(logits, 1)

    prediction = "Real" if predicted_class.item() == 1 else "AI"

    img_id = os.path.splitext(image_file)[0]

    data.append({"Id": img_id, "Label": prediction})

df = pd.DataFrame(data)
df.to_csv("predictions.csv", index=False)
